<a href="https://colab.research.google.com/github/TurkuNLP/ATP_kurssi/blob/master/Notenook9_2022_answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Please use the server to do these exercises. If you are unfamiliar with scripts, you can also start with the time-out exercises listed on Notebook8**

## Exercise 1

The Github repo https://github.com/MarkHershey/CompleteTrumpTweetsArchive has all the tweets published by Donald Trump when he was at office. Clone the repo to your home folder on the server.

### 1.1 
Count the most frequent hashtags and / or handles of the dataset covering Tweets when Trump was in office. Make sure to ignore possible punctuations, such as 
```
@realDonaldTrump:
@realDonaldTrump
```
### 1.2 
Make a script that takes as argument a handle and prints out its distribution over time month by month. Run the script on a couple of interesting handles / hashtags. Do you see any trends?

**Extra:** sort the output so that you have the tweets ordered from older to newer, followed by the number of tweets for that time stamp, like

```
YEAR-MONTH1 NUM-OF-TWEETS
YEAR-MONTH2 NUM-OF-TWEETS
```
If you get a permission denied error, you have forgotten to add execution rights to your script. This can be done with `chmod a+rwx file.txt`

### 1.3 
Make another script that takes as argument a handle and prints out a cleaned and normalized frequency list of the words that occur in the tweets with the handle. 

You can try out different ways of cleaning the data. Does it make sense to include tokens with numbers and / or punctuation at all? Or is it better to just, e.g., delete tokens and numbers and otherwise otherwise keep the strings there?

### 1.4 Advanced

The output of 1.3 would be much better if it excluded function words, don't you think?

Trankit is super slow and cannot be run on the server. But UdPipe can - the performance is lower, but maybe it would be enough. Let's try!

UdPipe documentation can be found at https://lindat.mff.cuni.cz/services/udpipe/api-reference.php 

With the following call, UdPipe prints to standard output the conllu analysis of the input file, here called delme.csv

`curl -F data=@delme.csv -F model=english -F tokenizer= -F tagger= -F parser= http://lindat.mff.cuni.cz/services/udpipe/api/process | PYTHONIOENCODING=utf-8 python -c "import sys,json; sys.stdout.write(json.load(sys.stdin)['result'])" `

If you have time, try to implement a script that is similar to 1.3 but outputs only content POS classes, such as nouns, verbs, etc, instead of all most frequent words, and lemmas instead of running words.

You can try with different POS classes to figure out the most useful output in your opinion. What happens if you include only nouns? or proper nouns? or adjetives? or all these classes?

### Tips
A new perl script can be useful for modifying strings. This example would replace all instances of REPLACED with ORIGINAL.

```
cat inputfile.txt | perl -pe 's/ORIGINAL/REPLACED/g'
```

This script supports also back reference, so you can refer to expressions in `ORIGINAL` and refer to those with `$` in `REPLACED`. Like here any punctuation is replaced by a newline and that punctuation

```
 perl -pe 's/([[:punct:]])/\n$1/g' 
```
----------------

Note that in a script, you can have various commands, and all the commands will be executed one by one (with the exception of pipes, where the commands are executed all at once).

For instance if a script has the two commands below, it will first print "kukkuu" to a file called delme.txt, and then, undepending of the first one, print the file and egrep lines correspomding to "kuu".

```
echo "kukkuu" > delme.txt

cat delme.txt | egrep "kuu"
```


## Some solutions

Note: pls make these on the course server. The scripts are also readable and runnable at 
/home/mavela/course-2022/notebook9-scripts


### 1.1




In [ ]:
! git clone https://github.com/MarkHershey/CompleteTrumpTweetsArchive.git

In [ ]:
%cd CompleteTrumpTweetsArchive/data/
! ls

In [ ]:
# to count the hashtags
# note that perl -pe 's/[[:punct:]]$//g' will replace any punctuation followed by a line ending with nothing

! cat realDonaldTrump_in_office.csv | cut -f 4 -d ',' | tr ' ' '\n' | egrep "#" | perl -pe 's/[[:punct:]]$//g' | sort | uniq -c | sort -rn | head


### 1.2

In [ ]:
# code to the script to count the time distribution of the tweets including a specific handle / hashtag, given as an argument
# so this could be run e.g. with cat tweets.csv | ./script.sh handle

! cat realDonaldTrump_in_office.csv | egrep -i $1 | cut -f 2 -d ',' | cut -f 2 -d ' ' | cut -f 1,2 -d '-'| sort | uniq -c


In [ ]:
# EXTRA:
#sort -k2 sorts the output by the second column, and the perl script with some regexes can be used to change the columns

! cat tweets.csv | ./script.sh handle | sort -k2 -n | perl -pe 's/^ *([0-9]+) ([0-9-]+)/$2 $1/g' 

In [ ]:
# the script runs in my folder /home/mavela/course-2022/notebook9-scripts on the server with
cat ../CompleteTrumpTweetsArchive/data/realDonaldTrump_in_office.csv | ./handlrefreqs.sh @WhiteHouse  | less


### 1.3 

In [ ]:
# code to count the most frequent words associated with a handle
# Note that this one ignores all rows - tokens - including a number or punctuation. I think this gave somewhat cleaner results.
# at least for @IvankaTrump I get words like *thank, thanks* and *commited, workforce, working* which hint the contents of these tweets

! cat realDonaldTrump_in_office.csv | egrep -i $1 | cut -f 4 -d ',' | tr ' ' '\n' | egrep -v '[[:punct:]]|[0-9]'  | tr '[[:upper:]]' '[[:lower:]]'| sort | uniq -c | sort -rn


In [ ]:
# the script runs in my folder /home/mavela/course-2022/notebook9-scripts on the server with

cat ../CompleteTrumpTweetsArchive/data/realDonaldTrump_in_office.csv | ./words_in_handle.sh @WhiteHouse  | less

### 1.4

In [ ]:
# the script runs in my folder /home/mavela/course-2022/notebook9-scripts on the server with

 cat ../CompleteTrumpTweetsArchive/data/realDonaldTrump_in_office.csv | ./conllu_in_handle.sh @WhiteHouse  | less
